In [19]:
import requests
from urllib.parse import urlencode
import uuid
import hashlib
import base64
from email.utils import formatdate
import json
from typing import Optional

class EDAN:
    edan_base = 'https://edan.si.edu/metadata/v2.0/collections/search.htm' 
    
    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key
        self.session = requests.Session()
    def query(self, 
              q: Optional[str] = None, fqs: Optional[dict] = None,
              start:int = 0, rows:int = 10):
        query_params = {'rows': rows,
                        'start': start}
        if fqs:
            fq_json = json.dumps([f'{k}:"{v}"' for k, v in fqs.items()])
            query_params['fqs'] = fq_json
        if q:
            query_params['q'] = q

        query_param_string = urlencode(query_params)
        
        request_date = formatdate()
        nonce = str(uuid.uuid4())
        string_to_sign = '\n'.join([nonce, query_param_string, request_date, 
                                    self.app_key])

        hashed_string = hashlib.sha1(string_to_sign.encode()).hexdigest()
        encoded_string = base64.b64encode(hashed_string.encode())
        request_headers = {'X-AppId': self.app_id,
                           'X-Nonce': nonce,
                           'X-RequestDate': request_date,
                           'X-AuthContent': encoded_string}

        r = self.session.get(self.edan_base, params = query_params, 
                             headers = request_headers)
        response_json = r.json()
        return response_json
    
    def fetch_all(self, 
                  q: Optional[str] = None, fqs: Optional[dict] = None, 
                  step:int = 100):
        results = []
        quick_result = self.query(q=q,fqs=fqs, rows=1)
        if 'rowCount' in quick_result:
            result_count = quick_result['rowCount']
        if result_count > 0:
            for start in range(0, result_count, step):
                edan_json = self.query(q=q,fqs=fqs, start=start, rows=step)
                edan_rows = edan_json['rows']
                results += edan_rows
        return results

In [20]:
with open('../edan_api_key.json') as api_json:
    edan_key = json.load(api_json)
edan = EDAN(edan_key['APP_ID'], edan_key['APP_KEY'])

In [21]:
fq_dict = {"unit_code":"NMAH",
           "online_media_type":"Images",
           "object_type":"real photo postcard"}

In [22]:
edan_r = edan.query(fqs=fq_dict, rows=1)

In [23]:
edan_r

{'rows': [{'id': 'edanmdm-nmah_1888444',
   'title': 'Two Men and a Woman Congregating Downtown',
   'unitCode': 'NMAH',
   'linkedId': '',
   'type': 'edanmdm',
   'url': 'edanmdm:nmah_1888444',
   'content': {'descriptiveNonRepeating': {'record_ID': 'nmah_1888444',
     'online_media': {'mediaCount': 2,
      'media': [{'thumbnail': 'https://ids.si.edu/ids/deliveryService?id=NMAH-AHB2018q010241&max=150',
        'idsId': 'NMAH-AHB2018q010241',
        'usage': {'access': 'Not determined'},
        'guid': 'http://n2t.net/ark:/65665/bq99ca746bd-f095-704b-e053-15f76fa0b4fa',
        'type': 'Images',
        'content': 'https://ids.si.edu/ids/deliveryService?id=NMAH-AHB2018q010241'},
       {'thumbnail': 'https://ids.si.edu/ids/deliveryService?id=NMAH-AHB2018q010242&max=150',
        'idsId': 'NMAH-AHB2018q010242',
        'usage': {'access': 'Not determined'},
        'guid': 'http://n2t.net/ark:/65665/bq99ca746bd-f096-704b-e053-15f76fa0b4fa',
        'type': 'Images',
        'conten

In [24]:
postcard_results = edan.fetch_all(fqs=fq_dict)
len(postcard_results)

1431

In [25]:
postcard_results[5]

{'id': 'edanmdm-nmah_1888540',
 'title': 'Man and woman Sitting on Beach Chairs by a Restaurant, Mountains in the Background',
 'unitCode': 'NMAH',
 'linkedId': '',
 'type': 'edanmdm',
 'url': 'edanmdm:nmah_1888540',
 'content': {'descriptiveNonRepeating': {'record_ID': 'nmah_1888540',
   'online_media': {'mediaCount': 2,
    'media': [{'thumbnail': 'https://ids.si.edu/ids/deliveryService?id=NMAH-AHB2018q010599&max=150',
      'idsId': 'NMAH-AHB2018q010599',
      'usage': {'access': 'Not determined'},
      'guid': 'http://n2t.net/ark:/65665/bq99ca746bd-a77b-704b-e053-15f76fa0b4fa',
      'type': 'Images',
      'content': 'https://ids.si.edu/ids/deliveryService?id=NMAH-AHB2018q010599'},
     {'thumbnail': 'https://ids.si.edu/ids/deliveryService?id=NMAH-AHB2018q010600&max=150',
      'idsId': 'NMAH-AHB2018q010600',
      'usage': {'access': 'Not determined'},
      'guid': 'http://n2t.net/ark:/65665/bq99ca746bd-a77c-704b-e053-15f76fa0b4fa',
      'type': 'Images',
      'content': 'ht

In [26]:
with open('postcard_edan_metadata.json','w') as json_out:
    json.dump(postcard_results, json_out)